In [2]:
# Author: Koji Kyoda
# Date: 19 June 2020
# sample code for writing BD5 data (circle entity)

import sys
import tables
from tables import *
import numpy as np
import cv2
import math
import warnings

# for supression of natural name warning
warnings.filterwarnings('ignore', category=tables.NaturalNameWarning)


# class definition
class ScaleUnit(IsDescription):
    dimension = StringCol(8, dflt="", pos=0)
    xScale = Float64Col(dflt=0, pos=1)
    yScale = Float64Col(dflt=0, pos=2)
    sUnit = StringCol(16, dflt="", pos=3)
    tScale = Float64Col(dflt=0, pos=4)
    tUnit = StringCol(16, dflt="", pos=5)

class ObjectDef(IsDescription):
    oID = UInt16Col(dflt=0, pos=0)
    name = StringCol(128, dflt="", pos=1)

class Data(IsDescription):
    ID = StringCol(32, dflt="", pos=0)
    t = Int32Col(dflt=0, pos=1)
    entity = StringCol(8, dflt="", pos=2)
    x = Float32Col(dflt=0, pos=3)
    y = Float32Col(dflt=0, pos=4)
    radius = Float32Col(dflt=0, pos=5)

# input and output
img_path = "./bw_sample.tif"
hdf5_out = "./sample_circle_bd5.h5"

# preprocessing
img = cv2.imread(img_path)
imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,127,255,0)

# find contours of ROIs
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#contours = list(filter(lambda x: cv2.contourArea(x) > 0, contours))
#contours = [np.squeeze(cnt, axis=1) for cnt in contours]

# output to bd5 file
h5file = open_file(hdf5_out, mode="w", title="bd5 file")
group = h5file.create_group('/', 'data', 'data group')

# for scaleUnit
scaleunit = h5file.create_table('/data', 'scaleUnit', ScaleUnit, "definition of scale and unit")
su = scaleunit.row
su['dimension'] = "2D"
su['xScale'] = 0.1
su['yScale'] = 0.1
su['sUnit'] = "micrometer"
su['tScale'] = 1
su['tUnit'] = "minute"
su.append()
scaleunit.flush()

# for objectDef
objectdef = h5file.create_table('/data', 'objectDef', ObjectDef, "object definition")
od = objectdef.row
od['oID'] = 0
od['name'] = "nucleus"
od.append()
objectdef.flush()

# series index
series = h5file.create_group('/data', '0', 'time index 0')

# object - create a HDF datagroup under time series index as object using pytable h5file.create_group
object0 = h5file.create_group('/data/0', 'object', 'object0')

# store xy coordinates to dataset table
table = h5file.create_table('/data/0/object', '0', Data, "dataset")
xyz = table.row
for i, cnt in enumerate(contours):
    M = cv2.moments(cnt)
    area = cv2.contourArea(cnt)
    radius = math.sqrt(area/math.pi) * 0.1
    cx = M['m10'] / M['m00']
    cy = M['m01'] / M['m00']
    xyz['ID'] = "o" + '%06d' % i
    xyz['t'] = 0
    xyz['entity'] = 'circle'
    xyz['x'] = cx
    xyz['y'] = cy
    xyz['radius'] = radius
    xyz.append()
table.flush()

h5file.close()

# how to read a dataset (test code)
# table2 = h5file.get_node('data', '0').read()

